In [1]:
# Imports
import plotly.express as px
from helper import *
import pandas as pd
import numpy as np
import gzip
import json
from scipy import stats
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Load Data

In [2]:
#Load mobility data 
mobility = get_mobility_df(get_country_dict('trust gov mobility')).reset_index()
mobility.head()


# Load and preprocess pageviews and deaths 
pageview_df = pd.read_csv("page_views_covid_related.csv.gz")
population_df = pd.read_csv("Population_countries.csv")

pageviews,_, _,_ = get_pageviews_df(pageview_df, population_df, get_country_dict('trust gov mobility'), '2020-01-22', '2022-07-31')
deaths, cases, _, _, _, _, _,_ = get_cases_deaths_df(population_df, get_country_dict('trust gov mobility'), '2020-01-22', '2022-07-31')

/Users/charlottsertic/Documents/GitHub/ada-2022-project-thedatadiggers22/helper.py:453: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  google_mobility = pd.read_csv(data_folder+'Global_Mobility_Report.csv.zip')


In [3]:
# Functions
def mobility_category(country, mobility):
    
    #Select the mobility for the specific country of interest
    df= mobility.loc[mobility['country_region']==country].drop(columns=['country_region_code', 'country_region','moving category', 'covid category'])
    df =  df.set_index('date')
    df = df.rename_axis("Mobility category", axis = "columns")

    #Create the plot
    fig = px.area(df, facet_col="Mobility category", facet_col_wrap=2)
    fig.update_layout( title="Mobility change per category for {} in %".format(str(country)))
    fig.show()


def covid_plot(country, deaths, cases, pageviews):
    dict_ = get_country_dict('trust gov mobility')
    df = pageviews[dict_[country]]
    df = pd.concat([df,cases[dict_[country]],deaths[dict_[country]]],axis = 1)
    cols = ['Pageviews', 'COVID cases', 'COVID deaths']
    df.columns = cols
    df = df.rename_axis("COVID related informations", axis = "columns")
    
    #Create the plot
    fig = make_subplots(rows=3, cols=1,shared_xaxes=True,vertical_spacing=0.02)
    fig.add_trace(go.Scatter(x=df.index, y=df['Pageviews'], name="Wikipedia pageviews"),row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['COVID cases'], name = "COVID cases"),row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['COVID deaths'], name = "COVID deaths"),row=1, col=1)
    fig.update_layout(height=600, width=600, title_text="The pandemic through cases, deaths and pageviews in {}".format(str(country)))
    fig.show()



# Case Studies
Analysis of 3 countries from our countries dataset, looking more in depth at the evolution of the pandemic over all our dataset metrics
First we look at how covid cases, deaths and wikipedia pageviews evolve in time. Then, we look at mobility by separating each category and plotting the evolution in time.  

## Case study: Germany 

In [9]:
covid_plot('Germany', deaths, cases, pageviews)

Looking at the plot above, one can see a clear distinction between pageviews cases and deaths. In fact, the number of pageviews increases sharply compared to the other metrics during the first months of the pandemic and then decreases with small rebounds of activity at each new wave. What is interesting is the comparison of deaths between the first and the second wave of the pandemic. It highlights the utility of the partial lockdown implemented on the 22nd of March 2020. Strangely, the two first wave of covid cases are very low compared to the third wave, which is expected as were not enough test made and available in Germany and thus the values were heavily underestimated. In the first part of the pandemic Germany successfully implemented contact tracing to quickly isolate cases and contain the dispersion, which we can see with lower covid cases until the 3rd wave. However, in 2022 there was a peak of covid cases as people were much more carefree about covid. Nonetheless, there were not as many deaths as in the 3rd wave as there were 77.8% of germans vaccinated. 




In [5]:
mobility_category('Germany', mobility)


During the pandemic Germany implemented a partial lockdown, forcing people to work at home and closing shops but still allowing them to go outside. This can be seen with the mobility evolution for each category plot above. In fact, categories such as Retail and Recreational, Workplace and Transit station endured a decrease from baseline. While Retail and Recreational and Transit station progressively returned to baseline over time, workplace stayed at a constant value below baseline. This is due to a restructuring of the way companies work, implementing more remote work. On the other hand, the park category remained constant at the beginning due to the partial lockdown, before a clear increase at the end of the lockdown, showing people had enough of staying inside and went out more. 


Partial lockdown because there is a mutual respect between the government and the people. The government didn't feel the need to force people, expected that they would be sensible and make the right choices. 

## Case study: Italy 

In [10]:
covid_plot('Italy', deaths, cases, pageviews)

Explanation:
The covid pandemic hit Italy very strongly, expecially the first wave which caused a lot of deaths, which can be seen on the covid deaths plot. We can also see that there is a large peak for covid pageviews exactly during the first wave, which shows that people heavily researched the virus as it arrived. However, the interest in the virus decreased over time which can be seen with the curve flattening. Simiarly to Germany, towards the end of the period, with 86.2% of the Italian population vaccinated, people started going out more and letting loose leading to a resurgence of a wave as seen in the covid cases plot. 




In [11]:
mobility_category('Italy', mobility)

Being so strongly hit by the virus, the government implemented a full lockdown on Italy on the 11th of March of 2020, forcing people to stay inside except for medical reasons or necesssary work. This lead to a tremendous decrease in mobility in Transit Stations, Parks, Workplace, Retail and Recreational. What is interesting is that the same trend was seen for Grocery and Pharmacy, which can be explained by people grocery shopping much less frequently as they were scared of getting infected. Contrary to Germany, there is an increase in Residential mobility from baseline, which can be explained by the full lockdown forcing people to stay home. We can observe phase 2 of the lockdown in Parks and Transit stations on the 4th of May, consisting of a relaxation of the of the mobility contraints in Italy, leading to a sharp increase in mobility. The large Park mobility spike happened on national day of Italy, leading to people celebrating outdoors. Finally, the lockdown being over one could think that mobility would go back to normal, however this is not observed for Parks and Workplace. 

## Case study: Thailand

In [12]:
covid_plot('Thailand', deaths, cases, pageviews)

Since european countries were in full lockdown, this decreased tourist traffic in Thailand allowing them to preserve low covid spread before a lockdown of their border on the 4th of April. Even though people were not touched by covid, the number of pageviews was important during the first european wave as people were documenting themselves about what was happening outside of thailand. Unlike Germany and Italy, the interest in covid remained high much longer until their first wave hit. It seems that there is a general curiosity on covid that remains until the actual spread of the virus in the country, which fades once it is experienced. We can see a strong relationship between covid deaths and cases, unlike Italy and germany.



In [13]:
mobility_category('Thailand', mobility)

Although covid did not hit Thailand in 2020 as hard as other countries, the government put in place a general curfew from 10pm to 4 am and a lockdown in densely populated regions.
That's why we observe a decrease in mobility for all categories but residential, which begin to go back to the norm at the end of the curfew on the 15th of June 2020. We also observe that Grocery and Pharmacy goes back to baseline around May 2020 and Transit station mobility remained below baseline. 
